# Project Description

Write a solution to:

- Find the name of the user who has rated the greatest number of movies. In case of a tie, return the lexicographically smaller user name.
- Find the movie name with the highest average rating in February 2020. In case of a tie, return the lexicographically smaller movie name.

The result format is in the following example.

## Example

### Input:

**Movies table:**
| movie_id    | title       |
|-------------|--------------|
| 1           | Avengers     |
| 2           | Frozen 2     |
| 3           | Joker        |

**Users table:**
| user_id     | name        |
|-------------|--------------|
| 1           | Daniel       |
| 2           | Monica       |
| 3           | Maria        |
| 4           | James        |

**MovieRating table:**
| movie_id | user_id | rating | created_at  |
|----------|---------|--------|-------------|
| 1        | 1       | 3      | 2020-01-12  |
| 1        | 2       | 4      | 2020-02-11  |
| 1        | 3       | 2      | 2020-02-12  |
| 1        | 4       | 1      | 2020-01-01  |
| 2        | 1       | 5      | 2020-02-17  | 
| 2        | 2       | 2      | 2020-02-01  | 
| 2        | 3       | 2      | 2020-03-01  |
| 3        | 1       | 3      | 2020-02-22  | 
| 3        | 2       | 4      | 2020-02-25  | 

### Output:
| results    |
|------------|
| Daniel     |
| Frozen 2   |

# Intuition

The problem likely involves analyzing movie ratings from a database. There are two main tasks:
1. **Find the user who has rated the most movies.** This involves counting how many movies each user has rated and then selecting the user with the highest count.
   
2. **Find the movie with the highest average rating in February 2020.** This task requires aggregating ratings for each movie but only for those ratings given in February 2020, then finding the movie with the highest average rating.

# Approach

- **First Query (User with Most Ratings):**
  - Use a `LEFT JOIN` to connect `Users` table with `MovieRating` table based on `user_id`. 
  - `GROUP BY u.user_id` to aggregate ratings per user.
  - `COUNT(mr.movie_id)` to count how many movies each user has rated.
  - `ORDER BY COUNT(mr.movie_id) DESC, u.name ASC` to sort users first by the number of ratings in descending order, then alphabetically by name. 
  - `LIMIT 1` to get only the top result.

- **Second Query (Movie with Highest Average Rating in February 2020):**
  - `LEFT JOIN` `Movies` with `MovieRating` on `movie_id`.
  - Filter ratings with `WHERE MONTH(mr.created_at) = 2 AND YEAR(mr.created_at) = 2020` to match February 2020.
  - `GROUP BY m.movie_id` to calculate average rating per movie.
  - `ORDER BY AVG(mr.rating) DESC, m.title ASC` to sort by average rating in descending order, then alphabetically by title.
  - `LIMIT 1` to fetch the top-rated movie.

# Complexity

- **Time complexity:** 
  - Both queries involve a JOIN operation which can be $$O(N \log N)$$ in the worst case due to sorting, where N is the number of ratings. However, with proper indexing, this might approach linear time $$O(N)$$.

- **Space complexity:**
  - O(N) as we might need to store all ratings temporarily during the join operations before aggregation.

# Code

```sql my sql
(SELECT u.name AS results
FROM Users u
LEFT JOIN MovieRating mr ON mr.user_id = u.user_id
GROUP BY u.user_id
ORDER BY COUNT(mr.movie_id) DESC, u.name ASC
LIMIT 1)

UNION ALL

(SELECT m.title AS results
FROM Movies m
LEFT JOIN MovieRating mr ON  m.movie_id = mr.movie_id
WHERE MONTH(mr.created_at) = 2 AND YEAR(mr.created_at) = 2020
GROUP BY m.movie_id
ORDER BY AVG(mr.rating) DESC, m.title ASC
LIMIT 1)

```
### In sql you have to do something like this to work
```sql
WITH TopUser AS (
    SELECT u.name AS results
    FROM Users u
    LEFT JOIN MovieRating mr ON mr.user_id = u.user_id
    GROUP BY u.user_id, u.name
    ORDER BY COUNT(mr.rating) DESC, u.name ASC
    LIMIT 1
),
TopMovie AS (
    SELECT m.title AS results
    FROM Movies m
    LEFT JOIN MovieRating mr ON mr.movie_id = m.movie_id 
    WHERE strftime('%Y-%m', mr.created_at) = '2020-02'
    GROUP BY m.movie_id, m.title
    ORDER BY AVG(mr.rating) DESC, m.title ASC
    LIMIT 1
)
SELECT * FROM TopUser
UNION
SELECT * FROM TopMovie;
